In [1]:
!pip install segmentation_models_pytorch

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
from mydataset import MyDataset
from tqdm import tqdm
from torch.cuda.amp import autocast
from data_augmentation import RandomRotation, RandomVerticalFlip, RandomHorizontalFlip, Compose, ToTensor
import segmentation_models_pytorch as smp

In [3]:
batch_size = 100
lr = 0.0001
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
train_set = MyDataset(root="data", is_train=True, transform=Compose([
    ToTensor(),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    RandomRotation([0, 90, 180, 270]),
]), normalize=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

val_set = MyDataset(root="data", is_train=False, transform=Compose([ToTensor()]),
                    normalize=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)
len(train_set), len(val_set), len(train_loader), len(val_loader)

(3800, 200, 38, 2)

In [5]:
model = smp.DeepLabV3Plus(
    in_channels=6,
    classes=2,
)
model.cuda()

criterion = nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode='max', factor=0.1, patience=2,
    verbose=True)
scaler = torch.cuda.amp.GradScaler(enabled=True)

In [6]:
def train_model(epoch):
    model.train()
    print(f"Epoch {epoch} Training")
    with tqdm(train_loader, desc=str(epoch)) as it:
        for idx, (img1, img2, mask) in enumerate(it, 0):
            img1, img2, mask = img1.cuda(), img2.cuda(), mask.cuda()
            optim.zero_grad()
            mask = mask.long()
            with autocast():
                img = torch.cat((img1, img2), 1)
                outputs = model(img)
                mask = mask.squeeze(1)
                loss = criterion(outputs, mask)
            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()
            _, pred = torch.max(outputs.data, 1)
            p, r, f1, iou = get_index(pred, mask)
            it.set_postfix_str(f"loss: {loss.item(): .4f} p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")

In [7]:
def get_index(pred, label):
    eps = 1e-7
    tp = torch.sum(label * pred)
    fp = torch.sum(pred) - tp
    fn = torch.sum(label) - tp

    p = (tp + eps) / (tp + fp + eps)
    r = (tp + eps) / (tp + fn + eps)
    f1 = (2 * p * r + eps) / (p + r + eps)
    iou = (tp + eps) / (tp + fn + fp + eps)
    return p, r, f1, iou


def test_model(epoch):
    model.eval()
    global max_score
    f1s = 0
    print(f"Epoch {epoch} Testing")
    with torch.no_grad():
        with tqdm(val_loader, desc=str(epoch)) as it:
            for img1, img2, mask in it:
                img1, img2, mask = img1.cuda(), img2.cuda(), mask.cuda()
                img = torch.cat((img1, img2), 1)
                outputs = model(img)
                _, pred = torch.max(outputs.data, 1)
                mask = mask.squeeze(1)
                p, r, f1, iou = get_index(pred, mask)
                f1s += f1
                it.set_postfix_str(f"p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")
    f1s /= len(val_loader)
    scheduler.step(f1s)
    print("f1", f1s.item())
    if max_score < f1s:
        max_score = f1s
        print('max_score', max_score.item())

In [8]:
num_epoch = 10
max_score = 0
for epoch in range(0, num_epoch):
    train_model(epoch=epoch)
    test_model(epoch=epoch)
print("completed!")
print('max_score', max_score.item())

Epoch 0 Training


0: 100%|██████████| 38/38 [00:54<00:00,  1.42s/it, loss:  0.3405 p:  0.7891  r:  0.6364  f1:  0.7046  iou:  0.5439]


Epoch 0 Testing


0: 100%|██████████| 2/2 [00:01<00:00,  1.58it/s, p:  0.7208  r:  0.7300  f1:  0.7254  iou:  0.5691]


f1 0.7401304693405258
max_score 0.7401304693405258
Epoch 1 Training


...

f1 0.8690778099449108
Epoch 9 Training


9: 100%|██████████| 38/38 [00:52<00:00,  1.37s/it, loss:  0.1127 p:  0.9009  r:  0.8916  f1:  0.8962  iou:  0.8120]


Epoch 9 Testing


9: 100%|██████████| 2/2 [00:01<00:00,  1.66it/s, p:  0.8970  r:  0.8544  f1:  0.8752  iou:  0.7781]

f1 0.8785498139132544
max_score 0.8785498139132544
completed!
max_score 0.8785498139132544
